In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from tqdm import trange
import numpy as np
import os
import shutil
from random import seed, shuffle
from PIL import Image

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

path = kagglehub.dataset_download("klu2000030172/birds-image-dataset")

print("Path to dataset files:", path)

In [5]:
# Set the directory paths
base_dir = '/Users/kailashkumar/Documents/CODE/bird-detector/data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Set the percentage for validation and test sets
val_pct = 0.20
test_pct = 0.10

In [9]:
# Function to count files and prepare distribution
def distribute_images(source_folder, dest_folder, fraction):
    # List all files in the source folder
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    shuffle(files)  # Shuffle for random distribution

    # Calculate number of files to move
    num_files_to_move = int(len(files) * fraction)
    files_to_move = files[:num_files_to_move]

    # Create destination folder if not exists
    os.makedirs(dest_folder, exist_ok=True)

    # Move files
    for file in files_to_move:
        shutil.move(os.path.join(source_folder, file), os.path.join(dest_folder, file))

    return len(files) - num_files_to_move, num_files_to_move

# Apply to both 'bird' and 'non-bird'
for category in ['bird', 'non-bird']:
    train_category_dir = os.path.join(train_dir, category)
    val_category_dir = os.path.join(val_dir, category)
    test_category_dir = os.path.join(test_dir, category)

    # Move to validation
    remaining, moved_to_val = distribute_images(train_category_dir, val_category_dir, val_pct)
    
    # Move to test
    _, moved_to_test = distribute_images(train_category_dir, test_category_dir, test_pct / (1 - val_pct))

    print(f"{category} - Moved {moved_to_val} to Validation, {moved_to_test} to Test. Remaining in Train: {remaining}")

bird - Moved 560 to Validation, 280 to Test. Remaining in Train: 2242
non-bird - Moved 560 to Validation, 280 to Test. Remaining in Train: 2241


In [14]:
# 1) Set up your directories
train_dir = 'data/train'
val_dir = 'data/val'
batch_size = 32
num_workers = 2  # might set to 0 or 1 on Pi if CPU usage is limited

# 2) Define image transformations
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# 3) Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir,   transform=val_transforms)

# 4) Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, 
                          shuffle=False, num_workers=num_workers)

# Inspect classes
print("Classes:", train_dataset.classes)  # should be ["bird", "not_bird"] if that’s your folder structure

Classes: ['bird', 'non-bird']


In [15]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

/Users/kailashkumar/Documents/CODE/bird-detector/myenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kailashkumar/Documents/CODE/bird-detector/myenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/kailashkumar/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 59.6MB/s]


In [16]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [20]:
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

In [22]:
num_epochs = 1
for epoch in range(num_epochs):  # number of epochs you want to train for
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        # Wrap this loop with tqdm for a progress bar
        for inputs, labels in tqdm(dataloaders[phase], desc=f'Epoch {epoch+1}/{num_epochs} - {phase}'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

Epoch 1/10 - train: 100%|██████████| 123/123 [00:46<00:00,  2.66it/s]


train Loss: 0.1801 Acc: 0.9314


Epoch 1/10 - val: 100%|██████████| 35/35 [00:20<00:00,  1.68it/s]


val Loss: 0.1363 Acc: 0.9509


Epoch 2/10 - train: 100%|██████████| 123/123 [00:46<00:00,  2.62it/s]


...

val Loss: 0.1338 Acc: 0.9527


Epoch 10/10 - train: 100%|██████████| 123/123 [00:46<00:00,  2.67it/s]


train Loss: 0.1382 Acc: 0.9495


Epoch 10/10 - val: 100%|██████████| 35/35 [00:20<00:00,  1.67it/s]

val Loss: 0.1213 Acc: 0.9589


In [23]:
torch.save(model.state_dict(), "bird_classifier.pth")

In [24]:
test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the images to the same size as training
    transforms.ToTensor(),  # Convert the images to Tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize like the others
])

test_dataset = datasets.ImageFolder(root='data/test', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [25]:
model.eval()  # Set the model to evaluation mode
test_loss = 0
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients for testing
    for data, target in tqdm(test_loader, desc='Testing Model'):
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item() * data.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

test_loss /= total
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Testing Model: 100%|██████████| 18/18 [00:16<00:00,  1.12it/s]

Test Loss: 0.1392, Test Accuracy: 95.36%


In [34]:
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = test_transforms(image)  # Apply the same transformations
    image = image.unsqueeze(0)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        return test_dataset.classes[predicted[0]]

# Example usage
image_path = 'data/final_checking/old-iron-flowerbed-feeders.jpeg'
prediction = predict_image(image_path)
print(f'Predicted: {prediction}')

Predicted: bird
